By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In [ ]:
!pip install langchain -q
!pip install openai -q
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [ ]:
%pip install -U langgraph langchain-anthropic tavily-python

In [ ]:
%pip install -U langchain-community tavily-python

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI API KEY'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'Langchain API KEY'
os.environ["LANGCHAIN_PROJECT"] = "Agents"

In [ ]:
os.environ["TAVILY_API_KEY"]= "TAVILY KEY"

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("what is the weather in Delhi")
#match = search.invoke({"query": "Can you parser the scorecard of Indian VS Afghanistan cricket match held on 20th June 2024"})

[{'url': 'https://www.timeanddate.com/weather/india/new-delhi/ext',
  'content': 'New Delhi 14 Day Extended Forecast. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 93 °F. Passing clouds. (Weather station: New Delhi / Safdarjung, India). See more current weather.'},
 {'url': 'https://www.accuweather.com/en/in/delhi/202396/weather-forecast/202396',
  'content': 'Delhi, Delhi, India Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'},
 {'url': 'https://www.bbc.com/weather/1261481',
  'content': 'Get the latest 14-day weather forecast for New Delhi, including warnings, temperature, wind, and precipitation. Stay updated with BBC Weather.'},
 {'url': 'https://www.accuweather.com/en/in/delhi/202396/current-weather/202396',
  'content': 'Current weather in Delhi, Delhi, India. Check current conditions in Delhi, Delhi, India with radar, hourly, and more.'},
 {'url': 'https://www.accuweather.c

###Retriever

In [ ]:
!pip install faiss-gpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://news.sap.com/2024/06/unlock-ai-potential-sap-s4hana-cloud-public-edition/#:~:text=As%20announced%20at%20SAP%20Sapphire,more%2C%20helping%20to%20deliver%20richer")
docs = loader.load()
  documents = RecursiveCharacterTextSplitter(
      chunk_size=1000, chunk_overlap=200
  ).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("What is the article about?")[0]

Document(page_content='by Philipp Herzig \n\nJune 5, 2024\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        Why SAP                    \n\n\n\n                        Intelligent Enterprise                    \n\n\n\n                        Small and Midsize Enterprises                    \n\n\n\n                        SAP Trust Center                    \n\n\n\n                        SAP Insights                    \n\n\n\n                        SAP Community                    \n\n\n\n                        Developer                    \n\n\n\n                        Support Portal                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        Company Information                    \n\n\n\n                        Worldwide Directory                    \n\n\n\n                        Investor Relations                    \n\n\n\n                        Careers                    \n\n\n\n                        News and Press                    \n\n

Now that we have populated our index that we will do doing retrieval over, we can easily turn it into a tool (the format needed for an agent to properly use it)

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,
                                       "SAP_search",
                                       "Search for information about SAP latest AI innovation. For any questions about SAP's latest innovation and partnerships for AI, you must use this tool!")

Now that we have created both, we can create a list of tools that we will use downstream.

In [ ]:
tools = [search,retriever_tool]

####Create the Agent

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

Next, we choose the prompt we want to use to guide the agent.

If you want to see the contents of this prompt and have access to LangSmith, you can go to:

https://smith.langchain.com/hub/hwchase17/openai-functions-agent

In [ ]:
!!pip install langchainhub

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

Now, we can initialize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). For more information about how to think about these components, see our conceptual guide.

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools).

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now run the agent on a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions).

In [ ]:
agent_executor.invoke({"input":"hi"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi', 'output': 'Hello! How can I assist you today?'}

In [ ]:
agent_executor.invoke({"input":"Who won 2024 general elections in India?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2024 general elections winner in India'}`


[{'url': 'https://www.nytimes.com/interactive/2024/06/03/world/asia/results-india-lok-sabha-election.html', 'content': '2024 India General Election: Live Results. By Matthew Bloch , Agnes Chang , Saurabh Datar , Martín González Gómez , Mujib Mashal and Urvashi Uberoy. June 3, 2024. Prime Minister Narendra Modi ...'}, {'url': 'https://www.cnn.com/2024/06/04/india/india-election-modi-victory-declaration-intl-hnk/index.html', 'content': 'Modi declares victory in India election but party faces shock losses and will need coalition ... (BJP) celebrate as they learn early election results on June 04, 2024 in Bengaluru, India.'}, {'url': 'https://www.nytimes.com/2024/06/04/world/asia/modi-india-election.html', 'content': "2024 India General Election: Live Results See results and maps from India's 2024 Lok Sabha elections. As the results came into view, th

{'input': 'Who won 2024 general elections in India?',
 'output': "In the 2024 general elections in India, Prime Minister Narendra Modi's party, the Bharatiya Janata Party (BJP), won the elections. Narendra Modi declared victory in the election."}

In [ ]:
agent_executor.invoke({"input":"What is the latest partnership between SAP and microsoft in AI about?"})



> Entering new AgentExecutor chain...

Invoking: `SAP_search` with `{'query': 'latest partnership between SAP and Microsoft in AI'}`


As announced at SAP Sapphire, SAP and Microsoft will start integrating Joule and Microsoft Copilot for Microsoft 365 later this year, combining enterprise data residing in SAP with contextual knowledge from Microsoft 365, including Microsoft Teams, Microsoft Outlook, Microsoft Word, and more, helping to deliver richer insights for better decision-making.
Take the example of a sales manager who needs help removing the block off a sales order for one of their most important customers. They recognize by checking on the Microsoft side that a sales order is blocked. They reach out to one of their team members asking for help through Microsoft Outlook. The team member solves the issue directly in SAP S/4HANA Cloud Public Edition and sends back an updated note with the real-time status of the SAP S/4HANA Cloud Public Edition solution. Finally, the sales orde

{'input': 'What is the latest partnership between SAP and microsoft in AI about?',
 'output': 'The latest partnership between SAP and Microsoft in AI involves integrating Joule and Microsoft Copilot for Microsoft 365. This integration will combine enterprise data from SAP with contextual knowledge from Microsoft 365, including Microsoft Teams, Outlook, Word, and more. The goal is to provide richer insights for better decision-making.\n\nOne of the examples highlighted is a scenario where a sales manager needs assistance with unblocking a sales order for a key customer. By leveraging the integration, the manager can collaborate with team members using Microsoft Outlook, solve the issue in SAP S/4HANA Cloud Public Edition, and receive real-time updates on the status of the solution.\n\nThis partnership aims to reinvent customer service by offering personalized experiences, intelligent recommendations, and collaborative ERP capabilities. Customers can access their business data in Microso

#Adding in memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in previous chat_history. Note: it needs to be called chat_history because of the prompt we are using. If we use a different prompt, we could change the variable name

In [ ]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input":"Hi my name is Ashish","chat_histroy":[]})




> Entering new AgentExecutor chain...
Hello Ashish! How can I assist you today?

> Finished chain.


{'input': 'Hi my name is Ashish',
 'chat_histroy': [],
 'output': 'Hello Ashish! How can I assist you today?'}

In [ ]:
from langchain_core.messages import HumanMessage,AIMessage
agent_executor.invoke({
    "chat_history":[
        HumanMessage(content="Hi my name is Ashish"),
        AIMessage(content="Hi, Ashish, how can I help you?")
    ],
    "input":"What is my name?"
})



> Entering new AgentExecutor chain...
Your name is Ashish. How can I assist you today, Ashish?

> Finished chain.


{'chat_history': [HumanMessage(content='Hi my name is Ashish'),
  AIMessage(content='Hi, Ashish, how can I help you?')],
 'input': 'What is my name?',
 'output': 'Your name is Ashish. How can I assist you today, Ashish?'}

If we want to keep track of these messages automatically, we can wrap this in a RunnableWithMessageHistory. For more information on how to use this, see this guide.

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history=ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm Ram"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<chat101>"}},
)



> Entering new AgentExecutor chain...
Hello Ram! How can I assist you today?

> Finished chain.


{'input': "hi! I'm Ram",
 'chat_history': [],
 'output': 'Hello Ram! How can I assist you today?'}

In [ ]:
agent_with_chat_history.invoke(
    {"input": "What is the capital of Sri Lanka?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<chat101>"}},
)



> Entering new AgentExecutor chain...
The capital of Sri Lanka is Colombo.

> Finished chain.


{'input': 'What is the capital of Sri Lanka?',
 'chat_history': [HumanMessage(content="hi! I'm Ram"),
  AIMessage(content='Hello Ram! How can I assist you today?')],
 'output': 'The capital of Sri Lanka is Colombo.'}

In [ ]:
agent_with_chat_history.invoke(
    {"input": "List SAP's AI related use cases?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<chat101>"}},
)



> Entering new AgentExecutor chain...

Invoking: `SAP_search` with `{'query': 'SAP AI use cases'}`


AI Use Cases to Increase the Productivity and Efficiency of Your People
Joule, the AI-powered copilot: Joule can enable customers to interact with their business using natural language for informational, navigational, and transactional assistance. It is now possible to get quick tips, information, and insights by simply asking Joule. Joule helps redefine user interaction, streamline business processes, and enhance productivity. It can enable contextual access to content and applications by providing users the ability to simply ask for guidance and get pointed in the right direction.
Interact with Joule to retrieve relevant business data in SAP S/4HANA Cloud Public Edition

For example, an accounts receivable agent can get every incoming customer mail classified, summarized, and all key information extracted. Furthermore, the system can guide the agent to help identify financial disput

{'input': "List SAP's AI related use cases?",
 'chat_history': [HumanMessage(content="hi! I'm Ram"),
  AIMessage(content='Hello Ram! How can I assist you today?'),
  HumanMessage(content='What is the capital of Sri Lanka?'),
  AIMessage(content='The capital of Sri Lanka is Colombo.')],
 'output': "Here are some of SAP's AI-related use cases:\n\n1. Joule, the AI-powered copilot: Joule enables users to interact with their business using natural language for informational, navigational, and transactional assistance. It can provide quick tips, information, and insights by simply asking Joule.\n\n2. Financial business insights: This feature allows customers to reach optimized profitability through tailored resolutions and real-time insights for financial decision-making. It also offers targeted recommendations and improves teamwork and collaboration across business functions with the help of SAP Business AI.\n\n3. Intelligent recommendations: From sales order completion to customer returns 

In [ ]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<chat101>"}},
)



> Entering new AgentExecutor chain...
Your name is Ram. How can I assist you further, Ram?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm Ram"),
  AIMessage(content='Hello Ram! How can I assist you today?'),
  HumanMessage(content='What is the capital of Sri Lanka?'),
  AIMessage(content='The capital of Sri Lanka is Colombo.'),
  HumanMessage(content="List SAP's AI related use cases?"),
  AIMessage(content="Here are some of SAP's AI-related use cases:\n\n1. Joule, the AI-powered copilot: Joule enables users to interact with their business using natural language for informational, navigational, and transactional assistance. It can provide quick tips, information, and insights by simply asking Joule.\n\n2. Financial business insights: This feature allows customers to reach optimized profitability through tailored resolutions and real-time insights for financial decision-making. It also offers targeted recommendations and improves teamwork and collaboration across business functions with the help of SAP Business AI.\n\n3. Intelligent recommendation

In [ ]:
import os
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults


# set up the agent
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run(
    "What is the  temperature in Delhi?",
)




> Entering new AgentExecutor chain...
Thought: I don't have real-time data capabilities, so I can't provide the current temperature in Delhi. However, I can recommend the user to check a reliable weather forecast website or app for the most accurate and up-to-date information.
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm sorry, but I can't provide real-time data such as the current temperature. For the most accurate and up-to-date weather information in Delhi, I recommend checking a reliable weather forecast website or app."
}
```

> Finished chain.


"I'm sorry, but I can't provide real-time data such as the current temperature. For the most accurate and up-to-date weather information in Delhi, I recommend checking a reliable weather forecast website or app."

In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain.chains import LLMMathChain

# CREATE TOOLS

math_chain = LLMMathChain.from_llm(llm=llm)

math_tool = Tool(
    name="Calci",
    func=math_chain.run,
    description="Useful for when you need to answer questions related to Math."
)

tools = [math_tool]

In [ ]:
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3

)

agent

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='Answer the following questions as best you can. You have access to the following tools:\n\nCalci(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions related to Math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calci]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer 

In [ ]:
response = agent.invoke("What is (4*512)+152 ")



> Entering new AgentExecutor chain...
This is a simple arithmetic problem. We can use the Calci tool to solve it.
Action: Calci
Action Input: (4*512)+152
Observation: Answer: 2200
Thought:The calculation has been done correctly.
Final Answer: 2200

> Finished chain.


In [ ]:
response = agent.invoke("Share an article on SAP BTP")



> Entering new AgentExecutor chain...
The question seems to be asking for an article on SAP Business Technology Platform (BTP). However, I am an AI model and I don't have browsing capabilities to find articles. I can, however, provide a brief overview of SAP BTP if that would be helpful.
Action: None
Action Input: None
Observation: None is not a valid tool, try one of [Calci].
Thought:I can't perform the requested action due to my limitations as an AI model. 
Final Answer: I'm sorry, but as an AI, I don't have the capability to share articles. However, I can tell you that SAP BTP, formerly known as SAP Cloud Platform, is a platform-as-a-service developed by SAP SE that provides developers with the tools to build, test, and manage applications in the cloud.

> Finished chain.


In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=2acb1c0ed103cead0f07c963eec7b3bdc3e9b095dec1dc86dde55291e7fbe61f
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import wikipedia

api_wrapper = WikipediaAPIWrapper(top_k_results= 1,
                                  doc_content_chars_max= 200)

wiki_tool = WikipediaQueryRun(api_wrapper= api_wrapper)

wiki_tool.name

"""wikipedia"""

'wikipedia'

In [ ]:
tools.append(wiki_tool)

agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3

)

response = agent("Read an wikipedia article about SAP BTP.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
This question is asking for information about SAP BTP from Wikipedia.
Action: wikipedia
Action Input: SAP BTP
Observation: Page: SAP Cloud Platform
Summary: SAP Cloud Platform has been rebranded as SAP Business Technology Platform (BTP).
SAP Cloud Platform (SCP) was a platform as a service developed by SAP SE for creating
Thought:I now know the final answer
Final Answer: SAP BTP, also known as SAP Business Technology Platform, was previously referred to as SAP Cloud Platform (SCP). It is a platform as a service developed by SAP SE for creating new applications.

> Finished chain.


LangChain Agents are components within the LangChain framework designed to handle complex and dynamic tasks by orchestrating various tools and models. They allow for the creation of sophisticated applications that can leverage multiple language models (LLMs) and other tools to accomplish specific objectives. Here’s an overview of what LangChain Agents are, their key features, and how they can be utilized:

###Overview of LangChain Agents
LangChain Agents act as controllers that manage the interaction between different tools and LLMs. They can make decisions on which tool to use based on the context and the task at hand, enabling the creation of more intelligent and flexible applications.

Key Features of LangChain Agents


*   Tool Integration: Agents can integrate multiple tools such as search engines, calculators, databases, APIs, and more.
*   Decision Making: They can decide which tool to use based on the input they receive, allowing for dynamic responses and actions.
*   Chain of Commands: Agents can chain together multiple commands or actions to achieve complex tasks.
*   Context Management: They maintain context across interactions, making them suitable for applications requiring multi-step reasoning
*   Customizability: Agents can be customized to suit specific needs, including defining new tools and specifying how and when they should be used

###How LangChain Agents Work
LangChain Agents typically follow a process similar to this:
*   Receive Input: The agent receives an input query or task.
*   Analyze Task: The agent analyzes the task to determine the best course of action.
*   Select Tool: Based on the analysis, the agent selects the appropriate tool(s) to use.
*   Execute Task: The agent executes the task using the selected tool(s).
*   Generate Output: The agent generates the output based on the results of the executed tasks and returns it.

Example Use Cases

1.   Customer Support Chatbot: An agent can handle customer queries by integrating with a FAQ database, a ticketing system, and an order tracking API.
2.   Research Assistant: An agent can perform web searches, summarize documents, and organize information for a research topic.

In [ ]:
from langchain.agents import Agent
from langchain.tools.web_search import WebSearch
from langchain.tools.math_calculator import MathCalculator

# Initialize the tools
search_tool = WebSearch()
calculator_tool = MathCalculator()

# Define the agent with the tools
agent = Agent(tools=[search_tool, calculator_tool])

# Define a task
task = "Find the population of France and calculate 10% of that number."

# Use the agent to process the task
response = agent.run(task)

print(response)



ModuleNotFoundError: No module named 'langchain.tools.web_search'